In [13]:
import pandas as pd
import numpy as np
import random
import sys
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneOut, cross_val_score
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## 1. Dataset Calling


In [14]:
class DatasetHelper:
    
    def __init__(self):
        df = pd.read_csv('datasetPF.csv')
        df = df[['project',
                'actual_effort',
                'size_ucp',
                'F1', 'F2','F3','F4','F5','F6','F7','F8']]
        self.datasets = df

    # return all dataset
    def all_datasets(self):
        data_array = self.datasets.to_dict(orient='index')

        return data_array

    # return all dataset - 1
    # leave one dataset from index
    def dataset_leave_one(self, index):
        leave_one = self.datasets.drop(index)
        leave_one = leave_one.reset_index(drop=True)
        leave_one = leave_one.to_dict(orient='index')
        return leave_one

In [15]:
# Test
loo = DatasetHelper()
loo.dataset_leave_one(119)

{0: {'project': 0,
  'actual_effort': 2124,
  'size_ucp': 118,
  'F1': 2,
  'F2': 2,
  'F3': 2,
  'F4': 2,
  'F5': 2,
  'F6': 3,
  'F7': 4,
  'F8': 0},
 1: {'project': 1,
  'actual_effort': 1430,
  'size_ucp': 130,
  'F1': 4,
  'F2': 3,
  'F3': 3,
  'F4': 4,
  'F5': 4,
  'F6': 3,
  'F7': 3,
  'F8': 0},
 2: {'project': 2,
  'actual_effort': 1445,
  'size_ucp': 85,
  'F1': 3,
  'F2': 3,
  'F3': 3,
  'F4': 3,
  'F5': 3,
  'F6': 4,
  'F7': 3,
  'F8': 0},
 3: {'project': 3,
  'actual_effort': 4895,
  'size_ucp': 275,
  'F1': 4,
  'F2': 4,
  'F3': 4,
  'F4': 5,
  'F5': 4,
  'F6': 4,
  'F7': 2,
  'F8': 0},
 4: {'project': 4,
  'actual_effort': 2420,
  'size_ucp': 110,
  'F1': 3,
  'F2': 2,
  'F3': 2,
  'F4': 3,
  'F5': 3,
  'F6': 3,
  'F7': 4,
  'F8': 0},
 5: {'project': 5,
  'actual_effort': 2080,
  'size_ucp': 65,
  'F1': 2,
  'F2': 2,
  'F3': 2,
  'F4': 2,
  'F5': 2,
  'F6': 3,
  'F7': 3,
  'F8': 4},
 6: {'project': 6,
  'actual_effort': 1265,
  'size_ucp': 55,
  'F1': 3,
  'F2': 3,
  'F3'

## 2. Bisecting K-Medoid

In [16]:
EXPONENT = 2

class BisectingKMedoid:
    
    def __init__(self, datasets):
        self.datasets = datasets
        self.clusters = []

    def random_medoid_tuple(self, arr_dataset):
        if (len(arr_dataset) == 1):
            return arr_dataset[0]

        random_index = random.randint(0, len(arr_dataset) - 1)
        
        return arr_dataset[random_index]

        
    def distance(self, arr_dataset, arr_tuple_of_medoids):
        ret_distance = {}
        for i in range(len(arr_dataset)):
            f1 = pow(arr_dataset[i]['F1'] - arr_tuple_of_medoids['F1'], EXPONENT)
            f2 = pow(arr_dataset[i]['F2'] - arr_tuple_of_medoids['F2'], EXPONENT)
            f3 = pow(arr_dataset[i]['F3'] - arr_tuple_of_medoids['F3'], EXPONENT)
            f4 = pow(arr_dataset[i]['F4'] - arr_tuple_of_medoids['F4'], EXPONENT)
            f5 = pow(arr_dataset[i]['F5'] - arr_tuple_of_medoids['F5'], EXPONENT)
            f6 = pow(arr_dataset[i]['F6'] - arr_tuple_of_medoids['F6'], EXPONENT)
            f7 = pow(arr_dataset[i]['F7'] - arr_tuple_of_medoids['F7'], EXPONENT)
            f8 = pow(arr_dataset[i]['F8'] - arr_tuple_of_medoids['F8'], EXPONENT)
            sumF = f1 + f2 + f3 + f4 + f5 + f6 + f7 + f8
            ret_distance.update({i: sumF})
        return ret_distance
    
    def variance(self, arr_distance):
        return sum(arr_distance.values()) / len(arr_distance)
    
        
    def create_two_cluster(self, arr_distance_c1, arr_distance_c2):
        ret = {'C1': [], 'C2': []}
        
        for key, val in arr_distance_c1.items():
            if val < arr_distance_c2[key]:
                ret['C1'].append(key)
            else:
                ret['C2'].append(key)
        
        return ret
    
    def bisecting_kmedoids_clustering(self):
        X = [self.datasets] # One set big cluster

        next_level = []
        V = X.copy() # One set big Cluster
        S = []
        arr_medoid_for_all_clusters = []


        while len(V) > 0:
            for val in V:
                if len(val) == 0:
                    break
                # print("VAL = ", val)
                arr_random_medoid_tuple_parent = self.random_medoid_tuple(val)
                parent_distance = self.distance(val, arr_random_medoid_tuple_parent)
                parent_variance = self.variance(parent_distance)

                # split menjadi dua cluster(C1, C2)
                arr_random_medoid_tuple_c1 = self.random_medoid_tuple(val)
                arr_random_medoid_tuple_c2 = self.random_medoid_tuple(val)
                arr_distance_c1 = self.distance(val, arr_random_medoid_tuple_c1)
                arr_distance_c2 = self.distance(val, arr_random_medoid_tuple_c2)
                variance_c1 = self.variance(arr_distance_c1)
                variance_c2 = self.variance(arr_distance_c2)
                two_cluster_c1_c2 = self.create_two_cluster(arr_distance_c1, arr_distance_c2)

                if max(variance_c1, variance_c2) < parent_variance:
                    temp = []
                    for vals in two_cluster_c1_c2:
                        for subval in two_cluster_c1_c2[vals]:
                            temp.append(val[subval])
                        next_level.append(temp)
                        temp = []
                else:
                    S.append(val)
                    arr_medoid_for_all_clusters.append(arr_random_medoid_tuple_parent)
                
            V = next_level
            next_level = []

        ret = {'medoids': arr_medoid_for_all_clusters, 'clusters': S}
        # print(ret['medoids'])
        # print("Cluster lenght: ", len(ret['clusters']))
        # print(ret['clusters'])
        self.clusters = ret
        return ret

    # create cluster name in datsets
    def clusters_naming(self):
        clusters =  self.clusters

        for i in range (len(clusters['clusters'])):
            for j in range (len(clusters['clusters'][i])):
                clusters['clusters'][i][j].update({"cluster": i})

        return clusters

In [17]:
# bisecting K-medoid with Leave One Out
# return data bisecting to csv

class ClusterGeneration:
    
    # leave one data
    def __init__(self, index):
        DatasetsClass = DatasetHelper()  #class call
        # return all datasets minus 1
        all_datasets = DatasetsClass.dataset_leave_one(index) 

        # running generate cluster
        self.bisecting_k_medoid_run(all_datasets)

        # check length cluster if less than 120 data
        clusters_len = pd.read_csv('clusters.csv')
        data_len = clusters_len.shape[0]
        while data_len < 119 :
            self.bisecting_k_medoid_run(all_datasets) # Generate cluster again

             # check again if length cluster if less than 119 data
            clusters_len = pd.read_csv('clusters.csv')
            data_len = clusters_len.shape[0]

    # calling bisecting k-medoid to generate cluster dataset
    def bisecting_k_medoid_run(self, all_datasets):


        # k_medoid class call
        k_medoid_class = BisectingKMedoid(all_datasets) 
        
        k_medoid_clustering = {}  
        k_medoid_clustering['clusters'] = []  # Initialize 'clusters' to an empty list
        while len(k_medoid_clustering['clusters']) < 2:  # repeat until there are more than 1 clusters
            k_medoid_clustering = k_medoid_class.bisecting_kmedoids_clustering() 

        named_clusters = k_medoid_class.clusters_naming() # clusters naming

        clusters_generated = []
        clusters_generated.append(named_clusters)

        # Save into csv
        datasets = named_clusters['clusters']

        # convert dataset train into one dimensional array
        data_dict = []
        for i in range(len(datasets)):
            for j in range(len(datasets[i])):
                data_dict.append(datasets[i][j])
                # print(datasets[i][j])

        # Construct DataFrame from dict
        df = pd.DataFrame.from_dict(data_dict) # data train
        df.to_csv('clusters.csv') # save to csv
        
        medoids = named_clusters['medoids']
        # Construct DataFrame from dict
        df_medoids = pd.DataFrame.from_dict(medoids) # data train
        df_medoids.to_csv('medoids.csv')


In [18]:
# TEST
# Runnning Cluster Generate minus one index
bisect = ClusterGeneration(119)

In [19]:
# # check length cluster
clusters_len = pd.read_csv('clusters.csv')
clusters_len.shape

(119, 13)

## 3. Support Vector Machine



In [20]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [21]:
class BisectingSupportVectorMachine:

    # svm to predicting cluster class
    def support_vector_machine(self, param_C, param_gamma, loo_index=1):

        df = pd.read_csv('clusters.csv')
  
        # split into x and y
        X = df.loc[:, ['F1', 'F2','F3','F4','F5','F6','F7','F8']] # data x
        y = df.loc[:, ['cluster']] # data y
        y = y.values.ravel() # convert into one dimensional array

        X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.25)
        unique_classes = np.unique(y_train)
        if len(unique_classes) <= 1: 
            # print(f"Only one class present in training data: {unique_classes}. Skipping SVM training.")
            ClusterGeneration(loo_index)
            X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.25)
            self.support_vector_machine(param_C, param_C, loo_index)
            

        # create a svm Classifier
        svm_classifier = SVC(kernel='rbf', C=param_C, gamma=param_gamma)

        # Train the model
        svm_classifier.fit(X_train, y_train)

        # Predict the response for test dataset
        y_pred = svm_classifier.predict(X_test)
        
        accuracy_score = metrics.accuracy_score(y_test, y_pred)

        # Predict cluster on data test
        df_test = pd.read_csv('datasetPF.csv')
        predicted_cluster = df_test.loc[df_test.project == loo_index, ['F1', 'F2','F3','F4','F5','F6','F7','F8']] # data x
        pred_cluster = svm_classifier.predict(predicted_cluster)[0]

        return accuracy_score, pred_cluster

In [22]:
svm = BisectingSupportVectorMachine()

accuracy_score, pred_cluster = svm.support_vector_machine(80.2, 2.1, loo_index=1)

print(accuracy_score, pred_cluster)

0.5333333333333333 0


## 4. Grey Wolf Optimizer

In [117]:
import sys, copy
import numpy as np

class GreyWolfOptimizer():
    def __init__(self, parameters, loo_index):
        self.parameters = parameters
        self.loo_index = loo_index
        self.fe = 0
        self.max_fe = 2 * 10000
    
    def isPositionOutOfRange(self, index, newPosition):
        for i in range(len(self.parameters)):
            if i == index and (newPosition < self.parameters['C'][0],self.parameters['g'][0] or newPosition > self.parameters['C'][1],self.parameters['g'][1]):
                return True
        
    def obj_func(self, x):
        svm = BisectingSupportVectorMachine() 
        accuracy_score, _ = svm.support_vector_machine(x[0], x[1], self.loo_index)
        self.fe += 1
        return accuracy_score
    
    def initialpop(self):
        population = []
        for i in range(self.parameters['popsize']):
            wolf = [random.uniform(self.parameters['C'][0], self.parameters['C'][1]),
                    random.uniform(self.parameters['g'][0], self.parameters['g'][1])]
            population.append(wolf)
        return population 
    
    def preyHunting(self, population, wolfType, C, A):
        for wolf in population:
            for i in range(len(wolfType)):
                D = abs(C * wolfType[i] - wolf[i])
                newPosition = wolfType[i] - A * D
                if self.isPositionOutOfRange(i, newPosition):
                    newPosition = wolfType[i]
                wolf[i] = newPosition
        return population
    
    def runGWO(self):
        bestWolf = {
            'wolfPositions': None,
            'ObjectValue': 0
        }
        bestConvergences = []
        population = self.initialpop()
        ob = np.array([self.obj_func(p) for p in population])
        sorted_indices = np.argsort(ob)[::-1]  # Sort indices of ob in descending order
        sorted_ob = ob[sorted_indices]
        best_ob = sorted_ob[:3]
        best_wolf = np.array(population)[sorted_indices][:3]
        
        # print(f"Object Value: {sorted_ob}")
        print(f"Top 3 Best ObjectValue: {best_ob}")
        # print("Parameters of Top 3 best Object Value:")
        print(f"Parameters: {best_wolf}")
    
        alphawolf = best_wolf[0]
        betawolf = best_wolf[1]
        deltawolf = best_wolf[2]

        for i in range(self.parameters['maxIter']):
            coefficient = 2
            a = coefficient - (coefficient * i) / self.parameters['maxIter']
            alphaA = (coefficient * a) * random.uniform(0, 1) - a
            betaA = (coefficient * a) * random.uniform(0, 1) - a
            deltaA = (coefficient * a) * random.uniform(0, 1) - a
            alphaC = coefficient * random.uniform(0, 1)
            betaC = coefficient * random.uniform(0, 1)
            deltaC = coefficient * random.uniform(0, 1)
            C = coefficient * random.uniform(0, 1)
            alphapopulation = self.preyHunting(population, alphawolf, C, alphaA)
            bestConvergences.append(bestWolf['ObjectValue'])
            if best_ob[0] > bestWolf['ObjectValue']:
                bestWolf['wolfPositions'] = alphawolf.tolist()
                bestWolf['ObjectValue'] = best_ob[0]
            betapopulation = self.preyHunting(population, betawolf, C, alphaA)
            deltapopulation = self.preyHunting(population, deltawolf, C, alphaA)
            
            for i in range (self.parameters['popsize']):
                positions = []
                for j in range (2):
                    position = (alphapopulation[i][j]+betapopulation[i][j]+deltapopulation[i][j])/2
                    positions.append(position)
                population[i] = positions
            print(bestWolf)   
            # sys.exit()
       
    
            
            

In [118]:
parameters = {'maxIter': 5, 
              'popsize': 10,
              'C': [0.01, 100],
              'g': [0.01, 50]}
print(parameters)
gwo = GreyWolfOptimizer(parameters, loo_index=1)
gwo.runGWO()

{'maxIter': 5, 'popsize': 10, 'C': [0.01, 100], 'g': [0.01, 50]}
Top 3 Best ObjectValue: [0.66666667 0.63333333 0.56666667]
Parameters: [[81.53665903 13.61100879]
 [57.8458875   4.69977087]
 [73.09386765 45.3442892 ]]
{'wolfPositions': [81.53665903346216, 13.611008786753416], 'ObjectValue': 0.6666666666666666}
{'wolfPositions': [81.53665903346216, 13.611008786753416], 'ObjectValue': 0.6666666666666666}
{'wolfPositions': [81.53665903346216, 13.611008786753416], 'ObjectValue': 0.6666666666666666}
{'wolfPositions': [81.53665903346216, 13.611008786753416], 'ObjectValue': 0.6666666666666666}
{'wolfPositions': [81.53665903346216, 13.611008786753416], 'ObjectValue': 0.6666666666666666}
